## Create data for the two $^*CO$ pathways that we don't have data on yet

### Create empty slabs 

In [26]:
import numpy as np
from ase.build import fcc111
from ase.db import connect
from ase.constraints import FixAtoms
import sys
#sys.path.append('../..')
#from shared_params import metals, lattice_parameters
experiment_path = "../Experiments/Formic_Acid_data/"

# Set random seed for reproducibility
np.random.seed(49226)

# Specify size of slab
size = (3,3,5)
n_atoms = np.prod(size)
n_atoms_surface = np.prod(size[:2])

# Set number of slabs to make
n_slabs = 56

# Connect to output database
with connect(experiment_path + 'slabs.db') as db:

	# Iterate through slabs
	for slab_idx in range(n_slabs):
	
		# Generate chemical symbols
		symbols = np.random.choice(metals, size=n_atoms)
	
		# Set the lattice parameter of the slab to the average of
		# the surface atoms' lattice constants
		surface_symbols = symbols[-n_atoms_surface:]
		lattice_param = sum(lattice_parameters[metal] for metal in surface_symbols) / n_atoms_surface
	
		# Make slab
		slab = fcc111('X', size=size, a=lattice_param, vacuum=10.)
	
		# Fix all but the two top layers of atoms
		constraint = FixAtoms(indices=[atom.index for atom in slab if atom.tag > 2])
		slab.set_constraint(constraint)
	
		# Set chemical symbols
		slab.set_chemical_symbols(symbols)
		
		# Save slab atoms object to database
		db.write(slab, slab_idx=slab_idx)


NameError: name 'metals' is not defined

### H on the side-reaction

In this reaction, I need two bound species:
* $^*COOH$
* $^*H$

on sites next to each other

In [24]:
import numpy as np
from ase.build import molecule
from ase.db import connect
from copy import deepcopy
import sys
sys.path.append('../..')
#from shared_params import metals #Send hjælp - hvor er denne?

# Specify size of slab
size = (3,3,5)
n_atoms = np.prod(size)
n_atoms_surface = np.prod(size[:2])

# Make CO adsorbate
CO = molecule('CO')
CO_orig = CO#[:-1]
CO_orig.rotate(-90, (1,0,0))

# Make OH adsorbate
OH = molecule('OH')
OH_orig = OH#[:-1]
OH_orig.rotate(-90, (1,0,0))

# Put carbon atom at (0,0,0)
C_pos = [atom.position for atom in CO_orig if atom.symbol == 'C'][0]
CO_orig.positions -= C_pos

# Put carbon atom at (0,0,0)
O_pos = [atom.position for atom in OH_orig if atom.symbol == 'O'][0]
OH_orig.positions -= O_pos

# Set height above surface to put adsorbate at
height = 1.9 # A

# Set number of slabs to make
n_slabs = 84

In [ ]:
# It seems this procedure requires having slabs already

In [ ]:
def add_CO_adsorbed(slab, site_idx=0):
    'Add COOH to slab at the specified site_idx'
    
    # Get position of on-top site
    site_position = slab[n_atoms - n_atoms_surface + site_idx].position
    
    # Set position of the adsorbate
    CO = deepcopy(CO_orig)
    CO.positions += site_position
    CO.positions[:, 2] += height

    # Add the adsorbate to the on-top site
    return slab + CO

def add_OH_adsorbed(slab, site_idx=0):
    'Add COOH to slab at the specified site_idx'

    # Get position of on-top site
    site_position = slab[n_atoms - n_atoms_surface + site_idx].position

    # Set position of the adsorbate
    OH = deepcopy(OH_orig)
    OH.positions += site_position
    OH.positions[:, 2] += height

    # Add the adsorbate to the on-top site
    return slab + OH

# Connect to output database
with connect('COOH_C_adsorbed.db') as db_ads,\
    connect('slabs.db') as db_slab:

    # Iterate through slabs
    for slab_idx in range(n_slabs):

        # Get slab from database
        slab = db_slab.get_atoms(slab_idx=slab_idx)

        # Iterate through surface on-top sites
        for site_idx in range(n_atoms_surface):

            # Get atoms with COOH adsorbed on carbon atom
            atoms = add_COOH_C_adsorbed(slab, site_idx)
            
            # Save atoms object to database
            db_ads.write(atoms, slab_idx=slab_idx, site_idx=site_idx)

# Add COOH to single element slabs
with connect('single_element_COOH_C_adsorbed.db') as db_ads,\
    connect('single_element_slabs.db') as db_slab:

    # Iterate through metals
    for metal in metals:

        # Get slab atoms object
        slab = db_slab.get_atoms(metal=metal)

        # Get atoms with COOH added
        atoms = add_COOH_C_adsorbed(slab)

        # Save atoms object to database
        db_ads.write(atoms, metal=metal)

### CO-OH slide-reaction

in this reaction, I need two bound species:
* $^*CO$
* $^*OH$

on sites next to each other.

The proposed mechanism is that $^*COOH$ splits and $^*OH$ slides onto the surface next to $^*CO$

### Here is an example of how Jack prepared the $^*COOH$ adsorbed data

In [7]:
import numpy as np
from ase.build import molecule
from ase.db import connect
from copy import deepcopy
import sys
sys.path.append('../..')
#from shared_params import metals

# Specify size of slab
size = (3,3,5)
n_atoms = np.prod(size)
n_atoms_surface = np.prod(size[:2])

# Make COOH adsorbate
HCOOH = molecule('HCOOH')
COOH_orig = HCOOH[:-1]
COOH_orig.rotate(-90, (1,0,0))

# Put carbon atom at (0,0,0)
C_pos = [atom.position for atom in COOH_orig if atom.symbol == 'C'][0]
COOH_orig.positions -= C_pos

# Set height above surface to put adsorbate at
height = 1.9 # A

# Set number of slabs to make
n_slabs = 56

def add_COOH_C_adsorbed(slab, site_idx=0):
	'Add COOH to slab at the specified site_idx'

	# Get position of on-top site
	site_position = slab[n_atoms - n_atoms_surface + site_idx].position

	# Set position of the adsorbate
	COOH = deepcopy(COOH_orig)
	COOH.positions += site_position
	COOH.positions[:, 2] += height

	# Add the adsorbate to the on-top site
	return slab + COOH

# Connect to output database
with connect('COOH_C_adsorbed.db') as db_ads,\
	 connect('slabs.db') as db_slab:

	# Iterate through slabs
	for slab_idx in range(n_slabs):
	
		# Get slab from database
		slab = db_slab.get_atoms(slab_idx=slab_idx)

		# Iterate through surface on-top sites
		for site_idx in range(n_atoms_surface):
			
			# Get atoms with COOH adsorbed on carbon atom
			atoms = add_COOH_C_adsorbed(slab, site_idx)
		
			# Save atoms object to database
			db_ads.write(atoms, slab_idx=slab_idx, site_idx=site_idx)

# Add COOH to single element slabs
with connect('single_element_COOH_C_adsorbed.db') as db_ads,\
	 connect('single_element_slabs.db') as db_slab:

	# Iterate through metals
	for metal in metals:
		
		# Get slab atoms object
		slab = db_slab.get_atoms(metal=metal)
		
		# Get atoms with COOH added
		atoms = add_COOH_C_adsorbed(slab)
		
		# Save atoms object to database
		db_ads.write(atoms, metal=metal)


KeyError: 'no match'

### Here is an example of how Jack prepares slabs

In [ ]:
import numpy as np
from ase.build import fcc111
from ase.db import connect
from ase.constraints import FixAtoms
import sys
sys.path.append('../..')
from shared_params import metals, lattice_parameters

# Set random seed for reproducibility
np.random.seed(49226)

# Specify size of slab
size = (3,3,5)
n_atoms = np.prod(size)
n_atoms_surface = np.prod(size[:2])

# Set number of slabs to make
n_slabs = 56

# Connect to output database
with connect('slabs.db') as db:

	# Iterate through slabs
	for slab_idx in range(n_slabs):
	
		# Generate chemical symbols
		symbols = np.random.choice(metals, size=n_atoms)
	
		# Set the lattice parameter of the slab to the average of
		# the surface atoms' lattice constants
		surface_symbols = symbols[-n_atoms_surface:]
		lattice_param = sum(lattice_parameters[metal] for metal in surface_symbols) / n_atoms_surface
	
		# Make slab
		slab = fcc111('X', size=size, a=lattice_param, vacuum=10.)
	
		# Fix all but the two top layers of atoms
		constraint = FixAtoms(indices=[atom.index for atom in slab if atom.tag > 2])
		slab.set_constraint(constraint)
	
		# Set chemical symbols
		slab.set_chemical_symbols(symbols)
		
		# Save slab atoms object to database
		db.write(slab, slab_idx=slab_idx)


In [29]:
reversed(list(range(1, 11)))